In [2]:
import os
import pandas as pd
import csv
import pre_proc_depth_id as pre

def open_csv(proc_file):
    # data = pre.data_to_proc(proc_file, proc_fields)
    # this is the consolidated csv just read each line
    data_b = os.path.join(os.getcwd(), "input", proc_file)
    return pd.read_csv(data_b)


# fields = ['date', 'lat', 'long', 'Depthm', 'T_degC', 'Salnty']
csv_data_new = open_csv("data_everything.csv")

In [10]:
csv_data_new.head(10)
csv_data_new.query('Depthm < 1')
csv_data_new.query('Depthm.between(80,90)')

,date,lat,long,Depthm,T_degC,Salnty
102,1949-3,38.179486,-127.461452,84,9.270,33.1000
193,1949-3,35.319012,-130.196135,87,12.660,33.2200
257,1949-3,35.727178,-128.673613,90,11.200,33.0800
321,1949-3,36.220640,-127.117569,82,11.370,32.9500
353,1949-3,36.380768,-126.271532,90,11.490,33.0100
...,...,...,...,...,...,...
864713,2016-1,31.179706,-120.919346,85,12.103,33.1877
864744,2016-1,30.846372,-121.589647,85,11.531,33.1794
864774,2016-1,30.513039,-122.257599,88,14.546,33.1970
864808,2016-1,30.179706,-122.923242,87,16.975,33.4372


In [12]:
subset = csv_data_new.query('Depthm < 1')
print(subset)

          date        lat        long  Depthm  T_degC   Salnty
0       1949-3  38.786665 -124.073108       0  10.500  33.4400
29      1949-3  38.499742 -125.729441       0  10.100  32.9500
61      1949-3  38.339614 -126.595201       0  10.200  32.6300
92      1949-3  38.179486 -127.461452       0  10.400  32.6800
123     1949-3  37.879486 -128.118349       0  10.500  32.7500
...        ...        ...         ...     ...     ...      ...
864731  2016-1  30.846372 -121.589647       0  17.923  33.2135
864762  2016-1  30.513039 -122.257599       0  17.920  33.1401
864795  2016-1  30.179706 -122.923242       0  19.390  33.3005
864827  2016-1  29.846372 -123.586614       0  19.556  33.2875
864858  2016-1  32.949052 -117.273565       0  18.744  33.4083

[33410 rows x 6 columns]


In [ ]:
df = csv_data_new
#cond_1 = df['Depthm'] < 1
cond_1 = df.Depthm < 1
cond_2 = df.date.str.startswith("1949")
#cond_3 = df.Salnty.isnull()
cond_3 = df.Salnty.notnull()
result = df.loc[cond_1 &  cond_2 & cond_3]
#result = df.loc[cond_2]



print(result)
print("++++++++++++++++")
print(len(result.index))

In [88]:
df = csv_data_new

depth_1 = 50
depth_2 = 100

cond_1 = df.Depthm.between(depth_1,depth_2)
cond_2 = df.date.str.startswith("1949")
cond_3 = (df.Salnty.notnull() & df.T_degC.notnull())
result = df.loc[cond_1 &  cond_2 & cond_3]




print(result)
print("++++++++++++++++")
print(len(result.index))

         date        lat        long  Depthm  T_degC  Salnty
7      1949-3  38.786665 -124.073108      50   10.24  33.424
8      1949-3  38.786665 -124.073108      58   10.06  33.420
9      1949-3  38.786665 -124.073108      75    9.86  33.494
10     1949-3  38.786665 -124.073108      78    9.83  33.510
11     1949-3  38.786665 -124.073108     100    9.67  33.580
...       ...        ...         ...     ...     ...     ...
28032  1949-1  33.903590 -120.702353     100    9.08  33.738
28060  1949-1  33.091154 -119.051503      50   11.16  33.070
28061  1949-1  33.091154 -119.051503      75   10.20  33.349
28062  1949-1  33.091154 -119.051503      78   10.09  33.370
28063  1949-1  33.091154 -119.051503     100    9.34  33.660

[4167 rows x 6 columns]
++++++++++++++++
4167


In [20]:
#csv_data_new.head(5)
s_min,s_max = csv_data_new['Salnty'].agg(['min','max'])
print(s_max)
print(s_min)

37.034
28.431


In [ ]:
import folium
import os



cali_map = folium.Map(location=[38.78666543,-124.0731082])

# csv_header = ['year', 'month', 'lat', 'long', 'Depthm', 'T_degC', 'Salnty']
for i, r in result.iterrows():
#for i, r in result.head(100).iterrows():
        #setting for the popup
    lat = r['lat']
    long = r['long']
    # make sure "lat" then "long"
    # silly mistake
    folium.Circle( (lat, long), 
                   radius=1,
                   legend_name = "blah"
                   ).add_to(cali_map)

cali_map

In [20]:
for file in os.listdir("output"):
    print(file)

1949.csv
1950.csv
1951.csv
1952.csv
1953.csv
1954.csv
1955.csv
1956.csv
1957.csv
1958.csv
1959.csv
1960.csv
1961.csv
1962.csv
1963.csv
1964.csv
1965.csv
1966.csv
1967.csv
1968.csv
1969.csv
1970.csv
1971.csv
1972.csv
1973.csv
1974.csv
1975.csv
1976.csv
1977.csv
1978.csv
1980.csv
1981.csv
1982.csv
1983.csv
1984.csv
1985.csv
1986.csv
1987.csv
1988.csv
1989.csv
1990.csv
1991.csv
1992.csv
1993.csv
1994.csv
1995.csv
1996.csv
1997.csv
1998.csv
1999.csv
2000.csv
2001.csv
2002.csv
2003.csv
2004.csv
2005.csv
2006.csv
2007.csv
2008.csv
2009.csv
2010.csv
2011.csv
2012.csv
2013.csv
2014.csv
2015.csv
2016.csv


In [ ]:
import folium
import pandas as pd
import folium.plugins
import math
import branca
import branca.colormap as cm

data = [
    [33.823400, -118.12194, 99.23],
    [33.823500, -118.12294, 95.23],
    [33.823600, -118.12394, 91.23],
    [33.823700, -118.12494, 90.00]
]

df = pd.DataFrame(data, columns=['latitude','longitude','power'])

x_start = (df['latitude'].max() + df['latitude'].min()) / 2
y_start = (df['longitude'].max() + df['longitude'].min()) / 2
start_coord = (x_start, y_start)


colormap = cm.LinearColormap(colors=['red','lightblue'], index=[90,100],vmin=90,vmax=100)

map = folium.Map(location=start_coord, zoom_start=12)


lat = list(df.latitude)
lon = list(df.longitude)
pow = list(df.power)


for loc, p in zip(zip(lat, lon), pow):
    folium.Circle(
        location=loc,
        radius=10,
        fill=True,
        color=colormap(p),
        #fill_opacity=0.7
    ).add_to(map)

map.add_child(colormap)

display(map)

In [ ]:
import folium
import pandas as pd
import folium.plugins
import math
import branca
import branca.colormap as cm

# building color scale 
#s_min,s_max = csv_data_new['Salnty'].agg(['min','max'])
s_min,s_max = result['T_degC'].agg(['min','max'])
# adjust scale
s_max = s_max + 1
s_min = s_min - 1

cali_map = folium.Map(location=[38.78666543,-124.0731082])
colormap = cm.LinearColormap(colors=['yellow','black'], index=[s_min,s_max],vmin=s_min,vmax=s_max)

# csv_header = ['year', 'month', 'lat', 'long', 'Depthm', 'T_degC', 'Salnty']
for i, r in result.iterrows():
        #setting for the popup
    lat = r['lat']
    long = r['long']
    #salt = r['Salnty']   
    salt = r['T_degC']
 
 
    # make sure "lat" then "long"
    # silly mistake
    folium.Circle( (lat, long), 
                   radius=1,
                   fill=True,
                   color=colormap(salt) 
                   ).add_to(cali_map)

cali_map.add_child(colormap)



In [90]:

def dots_on_a_map(df, target_inspection, year,depth):
    import folium
    import folium.plugins
    import branca.colormap as cm
    import os

    # builds map structure
    cali_map = folium.Map(location=[38.78666543, -124.0731082])

    # builds color scale
    s_min, s_max = df[target_inspection].agg(['min', 'max'])
    # adjust scale to add slight buffer either end
    s_max = s_max + 1
    s_min = s_min - 1
    
    colormap = cm.linear.YlOrBr_09.scale(s_min, s_max)
    colormap.caption = target_inspection + ' for '+ year + ' at ' + str(depth) + ' meters'

    for i, r in df.iterrows():
        # setting for the popup
        lat = r['lat']
        long = r['long']
        to_be_inspected = r[target_inspection]

        # make sure "lat" then "long"
        # silly mistake
        folium.Circle((lat, long),
                      radius=1,
                      fill=True,
                      color=colormap(to_be_inspected)
                      ).add_to(cali_map)

    # does not work inside of function but will work in notebook
    cali_map.add_child(colormap)
    
    # saving output as html 
    # saving map as png, has issues with zoom 
    fn= target_inspection + '_'+ year + '_'+ str(depth)  +'.html'
    cali_map.save(fn)
    
 
    
dots_on_a_map(result, 'Salnty','1949',depth_2)

